<a href="https://colab.research.google.com/github/daonly/personal/blob/main/ND_HRV_ICC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pingouin as pg
from openpyxl import Workbook

wb = Workbook()
ws = wb.active()

c_list = list(df.columns)
df_day = pd.read_excel(savepath+'\df_day.xlsx')
df_day['SubjectID'] = df_day['dataSource'].str[0:7]

for i in range(0, len(c_list)):
    t = pg.intraclass_corr(data=df_day,targets='SubjectID',raters='Day',ratings=c_list[i],nan_policy='omit')
    ws["B"+i+1] = t.loc[2][2]
    ws["C"+i+1] = t.loc[2][6]
    ws["D"+i+1] = t.loc[2][7]
    
wb.save(filename = savepath+'\ICC.xlsx')